Initialize the tracking procedure:

In [1]:
from tramway.analyzer import *

a = RWAnalyzer()

a.tracker.from_non_tracking()
a.tracker.dt = .04
a.tracker.localization_precision = .025
a.tracker.estimated_high_diffusivity = .3


/home/flaurent/github/RWA-python/rwa/hdf5.py:185: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel # Panel has been flagged deprecated


Retrieve the test data:

In [2]:
from tutorial import download_non_tracking_test_data
download_non_tracking_test_data()

Track a first localization data file:

In [3]:
trajectories = a.tracker.track('../data-examples/c04_high_density.tsf.xyt', register=True)

Track another localization data file:

In [4]:
trajectories = a.tracker.track('../data-examples/Manip02-01-Beta400AA-03-15ms.rpt.xyt', register=True)

IndexError: arrays used as indices must be of integer (or boolean) type

Check the trajectories are available in the analyzer for further processing:

In [ ]:
for f in a.spt_data:
    # the `trajectories` dataframes are registered in the central `spt_data` analyzer attribute
    print(f.dataframe)

Define further processing steps:

In [ ]:
a.tesseller = tessellers.KMeans
a.tesseller.resolution = .1 # optional; default value is determined from the average displacement length

a.sampler.from_nearest_neighbors(20) # optional; merely makes `a.sampler` visible

a.mapper.from_plugin('dd') # diffusivity and drift
# regularization always takes a lot of time...
#a.mapper.diffusivity_prior = 1.
#a.mapper.drift_prior = 1.

Process the tracked data and plot the resulting maps:

In [ ]:
from tramway.helper.inference import map_plot
import matplotlib.pyplot as plt

plot_options = dict(aspect='equal', unit='std', show=False)

for f in a.spt_data:
    
    # tessellate and assign translocations to bins
    sampling = a.sampler.sample(f.dataframe) # implicitly calls a.tesseller.tessellate
    
    # infer
    maps = a.mapper.infer(sampling)
    
    # plot
    fig, ax = plt.subplots(1,2,figsize=(16,8))
    map_plot(maps, sampling, feature='diffusivity', title='D', axes=ax[0], **plot_options)
    map_plot(maps, sampling, feature='drift', title='drift', axes=ax[1], **plot_options)
    